In [141]:
import os
import openai 

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

import warnings
warnings.filterwarnings("ignore")

In [142]:
import datetime
current_time = datetime.datetime.now().date()
target_time = datetime.date(2024, 6, 12)
if current_time > target_time:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [143]:
import pandas as pd
df = pd.read_csv("Data.csv")

In [144]:
df

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \n,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


### LLMChain

In [145]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [146]:
llm = ChatOpenAI(model_name=llm_model, temperature=0.9)

In [147]:
prompt = ChatPromptTemplate.from_template(
    "What is name of the biggest competitor that makes {product}?, give name only, no other text."
)

In [148]:
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True
)

In [149]:
product = df["Product"].iloc[1]
product

'Waterproof Phone Pouch'

In [150]:
chain.run(product=product)



> Entering new LLMChain chain...
Prompt after formatting:
Human: What is name of the biggest competitor that makes Waterproof Phone Pouch?, give name only, no other text.

> Finished chain.


'JOTO'

### SimpleSequentialChain

In [151]:
from langchain.chains import SimpleSequentialChain

In [152]:
llm = ChatOpenAI(model_name=llm_model, temperature=0.9)

first_prompt = ChatPromptTemplate.from_template(
    "What is name of the biggest competitor that makes {product}?, give name only, no other text."
)

chain1 = LLMChain(
    llm=llm,
    prompt=first_prompt,
    verbose=True
)     

In [153]:
sencond_prompt = ChatPromptTemplate.from_template(
    """Write a 20 words discription for the following competitors: {competitors}"""
)

chain2 = LLMChain(
    llm=llm,
    prompt=sencond_prompt,
    verbose=True
)


In [154]:
overall_chain = SimpleSequentialChain(
    chains=[chain1, chain2],
    # verbose=True
)   


In [155]:
overall_chain.run(product)



> Entering new LLMChain chain...
Prompt after formatting:
Human: What is name of the biggest competitor that makes Waterproof Phone Pouch?, give name only, no other text.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Write a 20 words discription for the following competitors: JOTO

> Finished chain.


'JOTO offers innovative office solutions with a focus on organization and productivity, perfect for professionals seeking efficiency and style.'

### SequentialChain

In [156]:
from langchain.chains import SequentialChain

In [157]:
llm = ChatOpenAI(model_name=llm_model, temperature=0.9)

first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to Vietnamese: {review}"
)

chain1 = LLMChain(
    llm=llm,
    prompt=first_prompt,
    verbose=False,
    output_key="vietnamese_review"
)   
second_prompt = ChatPromptTemplate.from_template(
    """What language is the following review: {vietnamese_review}"""
)

chain2 = LLMChain(
    llm=llm,
    prompt=second_prompt,
    verbose=False,
    output_key="language"
)

third_prompt = ChatPromptTemplate.from_template(
    """Summarize the following review in one sentence: {vietnamese_review} in {language} language"""
)

chain3 = LLMChain(
    llm=llm,
    prompt=third_prompt,
    verbose=False,
    output_key="vietnamese_summary"
)

fourth_prompt = ChatPromptTemplate.from_template(
        "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {vietnamese_summary}\n\nLanguage: {language}"
)

chain4 = LLMChain(
    llm=llm,
    prompt=fourth_prompt,
    verbose=False,
    output_key="follow_up_response"
)

overall_chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4],
    input_variables=["review"],
    output_variables=["vietnamese_review", "language", "vietnamese_summary", "follow_up_response"],
    verbose=False
)

review = df["Review"].iloc[1]
overall_chain(review)

{'review': 'I loved the waterproof sac, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on, once it was in the pouch.',
 'vietnamese_review': 'Tôi rất thích túi chống nước, mặc dù phần mở của nó được làm từ nhựa cứng. Tôi không biết liệu điều đó có dễ bị hỏng không. Nhưng khi điện thoại của tôi ở trong túi, tôi không thể bật máy được.',
 'language': 'This review is in Vietnamese.',
 'vietnamese_summary': 'The reviewer likes the waterproof bag, but is concerned about the hard plastic opening and the inability to use their phone while it is inside the bag.',
 'follow_up_response': 'Cảm ơn bạn đã chia sẻ ý kiến về túi chống nước. Chúng tôi rất vui khi biết bạn thích sản phẩm này. Chúng tôi sẽ tiếp thu ý kiến của bạn về việc mở túi bằng nhựa cứng và khả năng không sử dụng điện thoại khi đưa vào túi. Chúng tôi sẽ cố gắng cải thiện sản phẩm để đáp ứng nhu cầu của quý khách hàng. Cảm ơn bạn đã quan tâm và ủng hộ sản phẩm c

### Router Chain

In [158]:
datascience_template = """
You are a talented data scientist. You are great at answering data science related questions in a concise and easy to understand manner. \
When you don't know the answer, you will say 'I don't know'. \
You will not make up answers. \ 
Here is a question:
{input}"""

biology_template = """
You are a talented biology expert. You are great at answering biology related questions in a concise and easy to understand manner. \
When you don't know the answer, you will say 'I don't know'. \
You will not make up answers. \ 
Here is a question:
{input}"""

law_template = """
You are a talented lawyer. You are great at answering law related questions in a concise and easy to understand manner. \
When you don't know the answer, you will say 'I don't know'. \
You will not make up answers. \ 
Here is a question:
{input}"""

software_template = """
You are a talented software engineer. You are great at answering software engineering related questions in a concise and easy to understand manner. \
When you don't know the answer, you will say 'I don't know'. \
You will not make up answers. \
Here is a question:
{input} """

In [159]:
prompt_infos = [
    {
        "name" : "Data Science",
        "description": "Good for answering data science related questions",
        "template": datascience_template
    },
    {
        "name" : "Biology",
        "description": "Good for answering biology related questions",
        "template": biology_template
    },
    {
        "name" : "Law",
        "description": "Good for answering law related questions",
        "template": law_template
    },
    {
        "name" : "Software Engineering",
        "description": "Good for answering software engineering related questions",
        "template": software_template        
    }
]

In [160]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [161]:
llm = ChatOpenAI(model_name=llm_model, temperature=0)

In [162]:
destination_chains = {}
for prompt_info in prompt_infos:
    name = prompt_info["name"]
    prompt_template = prompt_info["template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=False
    )
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [163]:
destination_chains

{'Data Science': LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template="\nYou are a talented data scientist. You are great at answering data science related questions in a concise and easy to understand manner. When you don't know the answer, you will say 'I don't know'. You will not make up answers. \\ \nHere is a question:\n{input}"), additional_kwargs={})]), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000027738AC22B0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000027738AE2AF0>, root_client=<openai.OpenAI object at 0x0000027738B57370>, root_async_client=<openai.AsyncOpenAI object at 0x0000027738AC23A0>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********')), output_parser=StrOut

In [164]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(
    llm=llm,
    prompt=default_prompt,
    verbose=False
)

In [165]:
default_chain

LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000027738AC22B0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000027738AE2AF0>, root_client=<openai.OpenAI object at 0x0000027738B57370>, root_async_client=<openai.AsyncOpenAI object at 0x0000027738AC23A0>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={})

In [166]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of \
the candidate prompts listed below.\
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [167]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [168]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revisingit will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ "DEFAULT" or name of the prompt to use in Data Science: Good for answering data science related questions
Biology: Good for answering biology related questions
Law: Good for answering law related questions
Software Engineering: Good for answering software engineering related questions
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: The value of “destination” MUST match one of the candidate prompts listed below.If “destination” does not fit any of the specified prompts, set i

In [169]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [170]:
chain.run("What is a Huber Regression Model?")



> Entering new MultiPromptChain chain...
Data Science: {'input': 'What is a Huber Regression Model?'}
> Finished chain.


"A Huber regression model is a type of robust regression model that is less sensitive to outliers in the data compared to traditional least squares regression. It combines the best of both worlds by using a squared loss for data points close to the regression line and a linear loss for data points that are further away, effectively reducing the impact of outliers on the model's performance. This makes the Huber regression model more robust and reliable when dealing with noisy data."

In [171]:
chain.run("What is single cell RNA sequencing?")



> Entering new MultiPromptChain chain...
Biology: {'input': 'What is single cell RNA sequencing?'}
> Finished chain.


'Single-cell RNA sequencing is a technique used to analyze the gene expression profile of individual cells. It allows researchers to study the heterogeneity of gene expression within a population of cells, providing insights into cell types, cell states, and cellular functions. This technique has revolutionized our understanding of cellular diversity and has been instrumental in advancing fields such as developmental biology, cancer research, and immunology.'

In [140]:
chain.run("What is Tom and Jerry?")

'Tom and Jerry is a classic animated television series created by William Hanna and Joseph Barbera. It follows the comedic antics of a cat named Tom and a mouse named Jerry as they engage in a never-ending rivalry. The show first premiered in 1940 and has since become a beloved and iconic part of pop culture.'